## Load protobuf models
To build the protobuf files, I ran:
```
cd src;
protoc --python_out=. --proto_path=protobuf protobuf/sample.proto
```

In [17]:
import sys
from google.protobuf.json_format import MessageToJson
import tempfile
sys.path.append("../protobuf")

from sample_pb2 import Example, Type

## Create an example data record

In [8]:
example = Example(
    type = Type.FIZZ,
    content = "123123123"
)

In [9]:
# Note: if the value of id_field_type is set to 1 (default value) it will not be included in the payload
MessageToJson(example, including_default_value_fields=True)

'{\n  "content": "123123123",\n  "type": "FIZZ"\n}'

## Serialize the example record to prepare to store it in ipfs

In [10]:
content = example.SerializeToString()
content

b'\x12\t123123123'

## Try to deserialize it

In [12]:
reader = Example()
reader.ParseFromString(content)
MessageToJson(reader, including_default_value_fields=True)

'{\n  "content": "123123123",\n  "type": "FIZZ"\n}'

## Write the serialize message to ipfs

## Write the file that we want to upload
Long term it would be better to use HTTP or somehow write it in memory, but this looked like the easiest method

In [22]:
filename = "jupyter_test"
filepath = f"{tempfile.gettempdir()}/{filename}"
with open(filepath, "wb") as f:
    f.write(content)

## Upload the file to ipfs with the cli

In [23]:
import subprocess

In [24]:
subprocess.run(["ipfs", "files", "mkdir", "-p", "/data/"])


CompletedProcess(args=['ipfs', 'files', 'mkdir', '-p', '/data/'], returncode=0)

In [25]:
result = subprocess.run(["ipfs", "add", filepath, "--to-files", "/data/"], capture_output=True)
hash = result.stdout.strip()
hash

b'added QmdzCont2JQHCF6cFFtXeYySh5ewW1w9ra9SptfvLyLk9j jupyter_test'

## Read the data back from ipfs

In [26]:
result = subprocess.run(["ipfs", "files", "ls", "/data/"], capture_output=True)
data = result.stdout.strip()
data

b'Tjupyter_test\nidentity\njupyter_test\nloan\nvar'

In [27]:
result = subprocess.run(["ipfs", "files", "read", "/data/" + filename], capture_output=True)
data = result.stdout.strip()
data

b'\x12\t123123123'

In [29]:
reader2 = Example()
reader2.ParseFromString(data)
result = MessageToJson(reader2, including_default_value_fields=True)
result

'{\n  "content": "123123123",\n  "type": "FIZZ"\n}'

In [31]:
reader2.type

0

In [32]:
reader2.content

'123123123'